In [5]:
!wget http://dinamica10.fciencias.unam.mx/tmp/t2_1C_PB1-11_e41-81_mod_noise-04_451000.pat.txt

--2020-05-02 20:32:25--  http://dinamica10.fciencias.unam.mx/tmp/t2_1C_PB1-11_e41-81_mod_noise-04_451000.pat.txt
Resolving dinamica10.fciencias.unam.mx (dinamica10.fciencias.unam.mx)... 132.248.181.116
Connecting to dinamica10.fciencias.unam.mx (dinamica10.fciencias.unam.mx)|132.248.181.116|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235012634 (224M) [text/plain]
Saving to: ‘t2_1C_PB1-11_e41-81_mod_noise-04_451000.pat.txt’

t2_1C_PB1-11_e41-81 100%[===================>] 224.12M  11.1MB/s    in 21s     

2020-05-02 20:32:46 (10.9 MB/s) - ‘t2_1C_PB1-11_e41-81_mod_noise-04_451000.pat.txt’ saved [235012634/235012634]



In [0]:
#Importamos librerias utiles
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Dropout, Convolution2D, MaxPooling2D
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras


capas = keras.layers

In [0]:
#leemos el archivo que acabamos de descargar, omitiendo las primeras tres filas ya que esas no corresponden a los datos
datos = pd.read_csv("t2_1C_PB1-11_e41-81_mod_noise-04_451000.pat.txt", delimiter=';', skiprows=3,header=None)


In [8]:
z = np.where(datos[0]==0)
z[0].shape
#La primera columna del dataframe contiene puros ceros por lo que la vamos a eliminar

(451000,)

In [9]:
#tiramos la primera columna ya que son puros ceros y la columna 82 ya que eso se nos pidio en la tarea
datos = datos.drop([0,82], axis=1)
datos.rename(columns={81: "Clase"},inplace = True)
datos.sample(6)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,Clase
417962,0.00481,0.01008,0.01466,0.02004,0.03558,0.05988,0.08465,0.10823,0.12770,0.14584,0.18211,0.20733,0.22897,0.21628,0.22803,0.25310,0.26204,0.25650,0.25731,0.24371,0.23154,0.24718,0.23391,0.21814,0.23659,0.22630,0.23375,0.22229,0.23386,0.21350,0.21194,0.21794,0.20286,0.20880,0.21353,0.20316,0.20321,0.21148,0.19478,0.18939,...,0.17957,0.18808,0.19147,0.19475,0.17815,0.18535,0.19126,0.17165,0.18124,0.18602,0.18035,0.19000,0.18029,0.17899,0.19893,0.17951,0.18656,0.18429,0.17212,0.17507,0.16812,0.16507,0.17203,0.16819,0.16288,0.16794,0.15431,0.16525,0.15523,0.15831,0.15849,0.16044,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,418
14165,0.00935,0.01978,0.02987,0.04093,0.07416,0.12326,0.16558,0.21898,0.24459,0.30467,0.35770,0.41441,0.42214,0.45695,0.48793,0.49388,0.49614,0.50852,0.47429,0.47022,0.47026,0.44963,0.47870,0.47328,0.48186,0.46335,0.44104,0.43244,0.44133,0.43805,0.42004,0.43660,0.42556,0.42225,0.39063,0.43701,0.40600,0.42354,0.41901,0.37947,...,0.36836,0.39002,0.36784,0.39657,0.37706,0.37605,0.37859,0.38478,0.36851,0.38917,0.36712,0.38226,0.34161,0.37506,0.35580,0.35671,0.35133,0.33594,0.37386,0.33395,0.35413,0.33118,0.32237,0.32996,0.33358,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,15
425261,0.00514,0.00965,0.01483,0.02192,0.03697,0.06180,0.08278,0.10759,0.12350,0.14571,0.17318,0.21266,0.21465,0.23079,0.24444,0.24043,0.24498,0.24863,0.25158,0.23362,0.21022,0.22775,0.25262,0.22390,0.22757,0.22406,0.23359,0.22492,0.22710,0.22937,0.21809,0.22463,0.21464,0.21458,0.20910,0.19244,0.20497,0.19268,0.18653,0.20220,...,0.20168,0.18168,0.18947,0.18733,0.19780,0.17403,0.19439,0.18440,0.18293,0.17416,0.18257,0.18128,0.17552,0.20112,0.18908,0.17717,0.17933,0.18060,0.17013,0.16475,0.17675,0.18118,0.17425,0.16685,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,426
232150,0.00754,0.01627,0.02045,0.02945,0.05243,0.09317,0.12752,0.15504,0.20477,0.22737,0.28270,0.30157,0.32190,0.33684,0.35024,0.38448,0.37627,0.37321,0.35612,0.39249,0.33997,0.33442,0.35355,0.34275,0.33771,0.34656,0.32729,0.32559,0.33367,0.32914,0.31077,0.31133,0.33593,0.32605,0.33229,0.30436,0.31224,0.29020,0.27459,0.28478,...,0.28931,0.26748,0.28740,0.28318,0.29023,0.30471,0.26724,0.27116,0.26162,0.28047,0.28719,0.27896,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,233
369692,0.00557,0.01025,0.01553,0.02198,0.03957,0.07176,0.09537,0.11997,0.13942,0.17032,0.19505,0.22834,0.23584,0.23906,0.25350,0.26425,0.29159,0.28816,0.26433,0.26620,0.27474,0.27216,0.27409,0.24548,0.24012,0.25201,0.24772,0.25315,0.26713,0.24739,0.25077,0.24308,0.22920,0.21798,0.23421,0.23407,0.22116,0.22116,0.21901,0.23636,...,0.20986,0.22171,0.20751,0.21018,0.20142,0.20674,0.20632,0.21193,0.20492,0.20694,0.22052,0.21111,0.19520,0.20262,0.17933,0.18354,0.20238,0.20044,0.21282,0.19673,0.19105,0.18137,0.20654,0.19332,0.20540,0.19885,0.18881,0.19450,0.16968,0.15880,0.16897,0.16831,0.17372,0.16273,0.16793,0.17636,0.18212,0.17284,0.16601,370
407373,0.00506,0.01168,0.01488,0.02212,0.03896,0.07033,0.08933,0.11962,0.12971,0.16873,0.20115,0.23556,0.22894,0.24506,0.27097,0.25899,0.27714,0.28595,0.28032,0.26744,0.25414,0.26142,0.24553,0.25946,0.25653,0.25241,0.23978,0.24830,0.23823,0.23465,0.24028,0.23901,0.22191,0.23198,0.23642,0.22679,0.22214,0.22527,0.22064,0.20325,...,0.20534,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000

In [10]:
#contamos cuantas clases hay
print(datos.max(axis=0))
print(datos.min(axis=0))
#en total son 451 clases
datos['Clase'].value_counts()
#aparentemente las clases estan balanceadas.

1          0.01174
2          0.02326
3          0.03314
4          0.04665
5          0.08597
           ...    
77         0.35525
78         0.34022
79         0.34467
80         0.33663
Clase    451.00000
Length: 81, dtype: float64
1        0.00414
2        0.00810
3        0.01173
4        0.01628
5        0.03051
          ...   
77       0.00000
78       0.00000
79       0.00000
80       0.00000
Clase    1.00000
Length: 81, dtype: float64


255    1000
312    1000
327    1000
72     1000
328    1000
       ... 
408    1000
153    1000
409    1000
154    1000
256    1000
Name: Clase, Length: 451, dtype: int64

In [11]:
#dividimos datos en entrenamiento y prueba
entrenamiento, prueba = train_test_split(datos, test_size=0.25)
print(entrenamiento['Clase'].value_counts())
print(prueba['Clase'].value_counts())

280    785
124    784
282    782
302    781
258    780
      ... 
333    719
209    718
442    716
248    716
413    715
Name: Clase, Length: 451, dtype: int64
413    285
248    284
442    284
209    282
333    281
      ... 
258    220
302    219
282    218
124    216
280    215
Name: Clase, Length: 451, dtype: int64


In [0]:
#definimos conjuntos de entrenamiento y prueba (x e y)
xTrain = (entrenamiento.drop(columns=['Clase'])).to_numpy()
yTrain = (entrenamiento['Clase']).to_numpy()
xTest = (prueba.drop(columns=['Clase'])).to_numpy()
yTest = (prueba['Clase']).to_numpy()

In [53]:
#con una sola neurona de salida no es posible usar la funcion de perdida entropia cruzada categorica
#Este codigo se comenta porque arroja error, pero se deja para dejar el historico
#modeloS = Sequential()
#modeloS.add(Dense(80, input_shape=(80,),activation='relu'))
#modeloS.add(Dense(1000,activation='relu'))
#modeloS.add(Dense(1000,activation='relu'))
#modeloS.add(Dense(1,activation='softmax'))

#modeloS.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#modeloS.fit(xTrain,yTrain,epochs=5,batch_size=256)

Epoch 1/5


InvalidArgumentError: ignored

In [106]:
#Para tener una sola neurona de salida, es usamos la funcion de perdida mse. Sin embargo el rendimiento de este modelo no es nada bueno
modelo = Sequential()
#80 neuronas capa de entrada con 80 atributos de entrada para cada una
modelo.add(Dense(80, input_shape=(80,),activation='linear'))
#1000 neuronas totalmente conectadas con las siguientes mil y las neuronas de entrada. estas mil neuronas tienen funcion de activacion relu
modelo.add(Dense(1000,activation='relu'))
#1000 neuronas totalmente conectadas con las siguiente  y las neuronas anteriores. estas mil neuronas tienen funcion de activacion relu
modelo.add(Dense(1000,activation='relu'))
#una neurona de salida con funcion de activacion lineal
modelo.add(Dense(1,activation='linear'))

#compila modelo
modelo.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
#entrena red neuronal
modelo.fit(xTrain,yTrain,epochs=5,batch_size=128)

Epoch 1/5
338250/338250 [==============================] - 47s 140us/step - loss: 1940.5278 - accuracy: 0.0738
Epoch 2/5
338250/338250 [==============================] - 48s 141us/step - loss: 21.1597 - accuracy: 0.1138
Epoch 3/5
338250/338250 [==============================] - 48s 141us/step - loss: 18.8714 - accuracy: 0.1349
Epoch 4/5
338250/338250 [==============================] - 47s 140us/step - loss: 18.4499 - accuracy: 0.1403
Epoch 5/5
338250/338250 [==============================] - 47s 140us/step - loss: 16.5262 - accuracy: 0.1474


In [74]:
#ahora si ajustamos que en la capa de salida hayan 452 neuronas(clases) es posible entrenr usando la funcion de perdida
#entropia categorica y ademas el rendimiento del modelo es bastante bueno, habria que ver que sucede con los datos de prueba
modeloS = Sequential()
modeloS.add(Dense(80, input_shape=(80,),activation='linear'))
modeloS.add(Dense(2000,activation='relu'))
modeloS.add(Dense(2000,activation='relu'))
modeloS.add(Dense(452,activation='softmax'))

modeloS.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
modeloS.fit(xTrain,yTrain,epochs=5,batch_size=256)


Epoch 1/5
338250/338250 [==============================] - 138s 407us/step - loss: 1.0541 - accuracy: 0.7075
Epoch 2/5
338250/338250 [==============================] - 139s 412us/step - loss: 0.1161 - accuracy: 0.9673
Epoch 3/5
338250/338250 [==============================] - 138s 408us/step - loss: 0.0496 - accuracy: 0.9870
Epoch 4/5
338250/338250 [==============================] - 138s 407us/step - loss: 0.0355 - accuracy: 0.9905
Epoch 5/5
338250/338250 [==============================] - 138s 408us/step - loss: 0.0402 - accuracy: 0.9893


In [104]:
prediccion = modeloS.predict(xTest)
#define un nuevo vector a partir de las probabilidades tal que se redondee cada valor (0 o 1). esto se hace para poder calcular las metricas correctamente 
prediccion = np.rint(prediccion)
#Dado que el prediccion es una matriz y no un vector debemos transformar los valores predichos a sus respectivas clases esto se hace buscando la posicion de
#la clase mas probable (1) y se guarda un un vector llamado prediccion1 la posicion en la cual se encuentra dicha clase.
prediccion1 = np.zeros((prediccion.shape[0]),dtype=int)
for i in range(prediccion.shape[0]):
  z = np.where(prediccion[i]==1)
  prediccion1[i] = z[0][0]

print(accuracy_score(prediccion1,yTest))
#imprime tabla con los scores precision, recall, f1
print(classification_report(prediccion1,yTest))

0.999840354767184
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       241
           2       1.00      1.00      1.00       259
           3       1.00      1.00      1.00       247
           4       1.00      1.00      1.00       247
           5       1.00      1.00      1.00       223
           6       1.00      1.00      1.00       246
           7       1.00      1.00      1.00       235
           8       1.00      1.00      1.00       266
           9       1.00      1.00      1.00       262
          10       1.00      1.00      1.00       253
          11       1.00      1.00      1.00       258
          12       1.00      1.00      1.00       257
          13       1.00      1.00      1.00       262
          14       1.00      1.00      1.00       257
          15       1.00      1.00      1.00       228
          16       1.00      1.00      1.00       248
          17       1.00      1.00      1.00       245
         

In [0]:
#hacemos one hot encoiding a los datos y de entrenamiento para 452 clases
one_hot_labels = keras.utils.to_categorical(yTrain, num_classes=452)
#hacemos one hot encoiding a los datos y de prueba para 452 clases
labelsYVA = keras.utils.to_categorical(yTest, num_classes=452)
#con estos vectores ahora se va a entrenar una red neuronal

In [56]:
#red neuronal de prueba con dos capas, la de entrada tiene 80 neuronas y salida tiene 452 neuronas que corresponden a la clase
model = Sequential()
model.add(Dense(80, activation='relu', input_dim=80))
model.add(Dense(452, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(xTrain, one_hot_labels, epochs=10, batch_size=32)

Epoch 1/10
338250/338250 [==============================] - 18s 52us/step - loss: 2.7287 - accuracy: 0.2903
Epoch 2/10
338250/338250 [==============================] - 17s 50us/step - loss: 1.2043 - accuracy: 0.5651
Epoch 3/10
338250/338250 [==============================] - 17s 50us/step - loss: 0.9478 - accuracy: 0.6376
Epoch 4/10
338250/338250 [==============================] - 17s 50us/step - loss: 0.8212 - accuracy: 0.6791
Epoch 5/10
338250/338250 [==============================] - 17s 50us/step - loss: 0.7280 - accuracy: 0.7139
Epoch 6/10
338250/338250 [==============================] - 17s 51us/step - loss: 0.6511 - accuracy: 0.7444
Epoch 7/10
338250/338250 [==============================] - 17s 51us/step - loss: 0.5804 - accuracy: 0.7744
Epoch 8/10
338250/338250 [==============================] - 17s 51us/step - loss: 0.5167 - accuracy: 0.8023
Epoch 9/10
338250/338250 [==============================] - 17s 51us/step - loss: 0.4623 - accuracy: 0.8271
Epoch 10/10
338250/338250 [=

In [59]:
#red neuronal con dos capas ocultas de 2000 neuronas cada una. Una de entrada con 80 neuronas y una capa de salida con 452 neuronas que corresponden a cada clase
#Optimizador RMSProp
model = Sequential()
model.add(Dense(80, activation='relu', input_dim=80))
model.add(Dense(2000,activation='relu'))
model.add(Dense(2000,activation='relu'))
model.add(Dense(452, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# entrena modelo con 5 epocas
model.fit(xTrain, one_hot_labels, epochs=5, batch_size=32)

Epoch 1/5
338250/338250 [==============================] - 435s 1ms/step - loss: 0.9805 - accuracy: 0.6402
Epoch 2/5
338250/338250 [==============================] - 426s 1ms/step - loss: 0.2790 - accuracy: 0.9347
Epoch 3/5
338250/338250 [==============================] - 422s 1ms/step - loss: 0.2206 - accuracy: 0.9589
Epoch 4/5
338250/338250 [==============================] - 429s 1ms/step - loss: 0.1894 - accuracy: 0.9661
Epoch 5/5
338250/338250 [==============================] - 418s 1ms/step - loss: 0.1749 - accuracy: 0.9696


In [0]:
#predice las probabilidades de cada x de prueba
predicted = model.predict(xTest)

In [65]:
#define un nuevo vector a partir de las probabilidades tal que se redondee cada valor (0 o 1). esto se hace para poder calcular las metricas correctamente 
g= np.rint(predicted)
#imprime el accuracy score
print(accuracy_score(g,labelsYVA))
#imprime tabla con los scores precision, recall, f1
print(classification_report(g,labelsYVA))

0.9995920177383592
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00       242
           2       1.00      1.00      1.00       259
           3       1.00      1.00      1.00       248
           4       1.00      1.00      1.00       247
           5       1.00      0.94      0.97       238
           6       1.00      1.00      1.00       246
           7       1.00      1.00      1.00       235
           8       1.00      1.00      1.00       266
           9       1.00      1.00      1.00       263
          10       1.00      1.00      1.00       253
          11       1.00      1.00      1.00       259
          12       1.00      1.00      1.00       257
          13       1.00      0.98      0.99       266
          14       1.00      1.00      1.00       256
          15       1.00      1.00      1.00       228
          16       1.00      1.00      1.00       248
        

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
#red neuronal con dos capas ocultas de 2000 neuronas cada una. Una de entrada con 80 neuronas y una capa de salida con 452 neuronas que corresponden a cada clase
#dado que cada epoca tarda aproximadamente 10 minutos en terminar, reduciremos a 3 epocas. Para este caso usamos el optimizador Adadelta
model = Sequential()
model.add(Dense(80, activation='relu', input_dim=80))
model.add(Dense(2000,activation='relu'))
model.add(Dense(2000,activation='relu'))
model.add(Dense(452, activation='softmax'))
model.compile(optimizer='Adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# entrena modelo con 3 epocas
model.fit(xTrain, one_hot_labels, epochs=3, batch_size=32)

Epoch 1/3
338250/338250 [==============================] - 541s 2ms/step - loss: 1.1074 - accuracy: 0.6612
Epoch 2/3
338250/338250 [==============================] - 537s 2ms/step - loss: 0.1322 - accuracy: 0.9658
Epoch 3/3
338250/338250 [==============================] - 539s 2ms/step - loss: 0.0747 - accuracy: 0.9826


In [110]:
#predice las probabilidades de cada x de prueba
predicted = model.predict(xTest)
#define un nuevo vector a partir de las probabilidades tal que se redondee cada valor (0 o 1). esto se hace para poder calcular las metricas correctamente 
g= np.rint(predicted)
#imprime el accuracy score
print(accuracy_score(g,labelsYVA))
#imprime tabla con los scores precision, recall, f1
print(classification_report(g,labelsYVA))

0.9998669623059867
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      1.00      1.00       242
           2       1.00      1.00      1.00       259
           3       1.00      1.00      1.00       248
           4       1.00      1.00      1.00       247
           5       1.00      1.00      1.00       223
           6       1.00      1.00      1.00       246
           7       1.00      1.00      1.00       235
           8       1.00      1.00      1.00       266
           9       1.00      1.00      1.00       262
          10       1.00      1.00      1.00       253
          11       1.00      1.00      1.00       259
          12       1.00      1.00      1.00       257
          13       1.00      1.00      1.00       262
          14       1.00      1.00      1.00       256
          15       1.00      1.00      1.00       228
          16       1.00      1.00      1.00       248
        

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [111]:
#red neuronal con dos capas ocultas de 2000 neuronas cada una. Una de entrada con 80 neuronas y una capa de salida con 452 neuronas que corresponden a cada clase
#dado que cada epoca tarda aproximadamente 10 minutos en terminar, reduciremos a 3 epocas. Para este caso usamos el optimizador descenso del gradiente estocastico SDG
model = Sequential()
model.add(Dense(80, activation='relu', input_dim=80))
model.add(Dense(2000,activation='relu'))
model.add(Dense(2000,activation='relu'))
model.add(Dense(452, activation='softmax'))
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# entrena modelo con 3 epocas
model.fit(xTrain, one_hot_labels, epochs=3, batch_size=32)

Epoch 1/3
338250/338250 [==============================] - 284s 839us/step - loss: 5.3207 - accuracy: 0.0141
Epoch 2/3
338250/338250 [==============================] - 280s 829us/step - loss: 3.2665 - accuracy: 0.1327
Epoch 3/3
338250/338250 [==============================] - 279s 824us/step - loss: 1.8823 - accuracy: 0.3571


In [112]:
#predice las probabilidades de cada x de prueba
predicted = model.predict(xTest)
#define un nuevo vector a partir de las probabilidades tal que se redondee cada valor (0 o 1). esto se hace para poder calcular las metricas correctamente 
g= np.rint(predicted)
#imprime el accuracy score
print(accuracy_score(g,labelsYVA))
#imprime tabla con los scores precision, recall, f1
print(classification_report(g,labelsYVA))

0.20923281596452328
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.49      1.00      0.66       121
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.92      0.79      0.85       302
          10       0.00      0.00      0.00         0
          11       1.00      0.49      0.66       527
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       1.00      0.54      0.70       422
          16       0.00      0.00      0.00         0
       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Observamos que al usar el optimizador SDG para tres epocas, se obtiene un accuracy de 0.35 y una perdida de 1.88. Para los datos de prueba para este caso se obtiene un acuracy del 0.21.

Al intercambiar el optimizador por Adadelta se obtiene un acuracy de 0.9826 y una perdida de 0.07. Para los datos de prueba se obtiene un acuracy de 0.99986.

Al emplear el optimizador RMSprop se obtiene un acuracy para tres epocas de 0.9589 con una perdida de 0.22. Para los datos de prueba un acuracy de 0.99959.
Con esto vemos que el optimizador que fue mejor fue el Adadelta.
Notemos que con el SDG tenemos una perdida relaticamente grade y quizá al corrrer más epocas se podria llegar a obtener un rendimiento mejor.
